In [16]:
#import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from pprint import pprint

In [ ]:
# load ascii text and covert to lowercase
filename = "FAHRENHEIT 451.txt"
raw_text = open(filename, 'r').read()
#raw_text = raw_text.lower()
print(raw_text)

In [18]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, '`': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74}


In [19]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  251222
Total Vocab:  75


In [20]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 80
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  251142


In [21]:
print(dataX[1])

[23, 30, 40, 27, 36, 30, 27, 31, 42, 1, 14, 15, 11, 1, 50, 73, 1, 40, 49, 73, 1, 24, 66, 49, 52, 50, 69, 66, 73, 0, 42, 56, 57, 67, 1, 63, 62, 53, 7, 1, 71, 57, 68, 56, 1, 55, 66, 49, 68, 57, 68, 69, 52, 53, 7, 1, 57, 67, 1, 54, 63, 66, 1, 26, 37, 36, 1, 25, 37, 36, 29, 26, 37, 36, 9, 0, 28, 23, 30, 40]


In [22]:
print(dataY[1])

27


In [23]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512,input_shape=(X.shape[1], X.shape[2]), return_sequences=True)) 
#model.add(LSTM(512,input_shape=(X.shape[1], X.shape[2]))) 
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-128->256.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=16, batch_size=8, callbacks=callbacks_list)

Epoch 1/16
17382/31393 [===============>..............] - ETA: 15:30 - loss: 2.8071